<h1 align="center">Multi Player Perceptron</h1>

## Integrantes

- Josué Say
- André Jo

## Repositorio

- [Enlace a GitHub](https://github.com/JosueSay/labs_dl)

## Instalar dependecias 

In [ ]:
# %pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Librerias

In [1]:
import numpy as np
import altair as alt 
import pandas as pd

In [2]:
def init_parameters(n_features, n_neurons, n_output): 

    np.random.seed(100)
    W1 = np.random.uniform(size = (n_features, n_neurons))
    b1 = np.random.uniform(size = (1, n_neurons))

    W2 = np.random.uniform(size = (n_neurons, n_output))
    b2 = np.random.uniform(size = (1, n_output))

    return {
        "W1" : W1 
        , "b1" : b1 
        , "W2" : W2
        , "b2" : b2 
    }

Esta función inicializa los parametros para una red neuronal recibiendo la cantidad de entradas que se desea, cantidad de neuronas y las salidas. Dando el caso para la función sort tendriamos como entrada $X_1$ y $X_2$ para una poperación XOR con una salida $Y$, esto indica que pereparamremos una salida con pesos de n_features filas y `n_neurons` columnas, para el caso de bias un matriz de 1 fila y `n_neurons` columnas para el caso de los primeros bias y pesos. para el segundo se hace una preparación similar pero con un orden inverso para que concuerde en la operación de estos valores usando `n_neurons` para filas y `n_output` para columnas y para el bias 1 fila con `n_output` columnas.

In [3]:
def linear_function(W, X, b): 
    return (X @ W)+ b 

Esta función hace la parte lineal de la neurona.

$$
z = \sum_i x_i w_i + b
$$

In [4]:
def sigmoid_func(Z): 
    return 1 / (1 + np.exp(-Z))

Es la función de activación sigmoide, y es la parte no lineal del modelo.

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

In [5]:
def cost_function(A, y): 
    return (np.mean(np.power(A - y, 2)))/2 

In [6]:
def predict(X, W1, W2, b1, b2): 
    Z1 = linear_function(W1, X, b1)
    S1 = sigmoid_func(Z1)
    Z2 = linear_function(W2, S1, b2)
    S2 = sigmoid_func(Z2)
    return np.where(S2 >= 0.5, 1, 0)

In [7]:
def fit(X, y, n_features = 2, n_neurons = 3, n_output = 1, iterations = 10, eta = 0.001): 

    params = init_parameters(
        n_features= n_features
        , n_neurons= n_neurons
        , n_output= n_output
    )

    errors = [] 

    for _ in range(iterations): 

        Z1 = linear_function(params['W1'], X, params['b1'])
        S1 = sigmoid_func(Z1)
        Z2 = linear_function(params['W2'], S1, params['b2'])
        S2 = sigmoid_func(Z2)

        error = cost_function(S2, y)
        errors.append(error)

        delta2 = (S2 - y) * S2 * (1 - S2)
        W2_gradients = S1.T @ delta2 
        params["W2"] = params["W2"] - W2_gradients * eta

        params["b2"] = params["b2"] - np.sum(delta2, axis = 0, keepdims= True) * eta 

        delta1 = (delta2 @ params["W2"].T) * S1 * (1 - S1)
        W1_gradients = X.T @ delta1 
        params["W1"] = params["W1"] - W1_gradients * eta 

        params["b1"] = params["b1"] - np.sum(delta1, axis = 0, keepdims= True) * eta 

    return errors, params 


        

In [8]:
y = np.array([[0, 1, 1, 0]]).T 
X = np.array([[0, 0, 1, 1]
              ,[0, 1, 0, 1]]).T 

In [9]:
errors, params = fit(X, y, iterations=5000, eta = 0.1)

In [10]:
y_pred = predict(X, params["W1"], params["W2"], params["b1"], params["b2"])
num_correct_predictions = (y_pred == y).sum()
accuracy = (num_correct_predictions / y.shape[0]) * 100
print('Multi-layer perceptron accuracy: %.2f%%' % accuracy)


Multi-layer perceptron accuracy: 100.00%


In [11]:
alt.data_transformers.disable_max_rows()
df = pd.DataFrame({"errors":errors, "time-step": np.arange(0, len(errors))})
alt.Chart(df).mark_line().encode(x="time-step", y="errors").properties(title='Chart 2')


alt.Chart(...)

# Mismo Dataset en Framework Pytorch

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import altair as alt

In [13]:
# Inputs: XOR problem
X = np.array([[0, 0, 1, 1],
              [0, 1, 0, 1]]).T  # Shape (4, 2)
y = np.array([[0, 1, 1, 0]]).T  # Shape (4, 1)

# Convert to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Define the model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = SimpleNet()

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
errors = []
epochs = 3000

for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    errors.append(loss.item())

# Convert errors to DataFrame and plot
df2 = pd.DataFrame({"errors": errors, "time-step": np.arange(epochs)})

alt.Chart(df2).mark_line().encode(
    x="time-step",
    y="errors"
).properties(title='Chart 3')


alt.Chart(...)

## Preguntas

### ¿Existen cambios de arquitectura en las 2 redes implementadas?

*Respuesta*

### ¿Exiten diferencias en la velocidad de convergencia entre las 2 redes?

*Respuesta*

### En la función init_parameters, ¿qué sucede si inicializamos los pesos en 0 en lugar de valores aleatorios? ¿y los bias?

*Respuesta*